In [27]:
#%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import log_loss 
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import warnings
warnings.simplefilter(action='ignore')

In [28]:
test = pd.read_csv('test_label_enc.csv')
train = pd.read_csv('train_label_enc.csv')

In [29]:
train.Stage.value_counts()

1.0    9529
0.0    7315
Name: Stage, dtype: int64

In [30]:
# Aca saco las columnas que no quiero que las tome mi algoritmo.
features = list(set(train.columns) - set(['ID','Stage','Sales_Contract_No','contrato','Opportunity_ID','Opportunity_Created_Date','Account_Created_Date','Opportunity_Name',\
                                        'Price', 'Currency', 'ASP', 'ASP_converted_Currency', 'ASP_(converted)',\
                                        'Planned_Delivery_Start_Date','Size', 'Planned_Delivery_End_Date', 'Month',\
                                         'Delivery_Year', 'Actual_Delivery_Date','Product_Type','Product_Category_B',\
                                        'Total_Amount_Currency', 'Total_Amount','Last_Activity',\
                                        'Quote_Expiry_Date', 'Last_Modified_Date',\
                                        'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount','Account_Created_Date'\
                                          ,'Total_moneda_local','ASP_Currency','Last_Modified_By','Brand','Bureaucratic_Code',\
                                         'tiempo_creacion_ulti_mod','ASP_converted','Product_Name','Opportunity_Owner',\
                                          'Account_Owner','Product_Family','Account_Name','Estacion_entrega','Source']))
#features

In [31]:
train_pos= train.loc[train['Stage']==1,:].sample(7315)
train_neg = train.loc[train['Stage']==0,:].sample(7315)
train=pd.concat([train_pos,train_neg]).drop_duplicates(keep=False)
train= shuffle(train)

In [32]:
X = train[features]
Y = train['Stage']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [33]:
gbm = LGBMClassifier()

In [34]:
gbm.fit(x_train[features], y_train)

LGBMClassifier()

In [35]:
scores = cross_val_score(gbm, train[features], train['Stage'], cv=3,scoring= 'neg_log_loss')
scores

array([-0.22941645, -0.22799534, -0.23447693])

In [36]:
abs(scores.mean())

0.2306295743150452

In [163]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = True):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2)
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)[:,1]
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [164]:
model = lgb.LGBMClassifier()
param_grid = {
    'n_estimators': [10,50, 90],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [15,20,25],
    'num_leaves': [50, 100, 200],
}

model, pred = algorithm_pipeline(x_train, x_test, y_train, y_test, model, 
                                 param_grid, cv=5, scoring_fit='neg_log_loss')

print(model.best_score_)
print(model.best_params_)

In [37]:
x_test['prediccion']=gbm.predict_proba(x_test[features])[:,1]

In [38]:
x_test['real'] = y_test

In [39]:
log_loss( x_test['real'].astype('float'),x_test['prediccion'].astype('float'))

0.22283661893953893

In [40]:
gbm.fit(train[features], train['Stage'])

LGBMClassifier()

In [41]:
pred = gbm.predict_proba(test[features])[:,1]

In [42]:
len(pred)

2551

In [43]:
test['target']=pred

In [44]:
test[['Opportunity_ID', 'target']]

,Opportunity_ID,target
0,10689,0.526405
1,10689,0.526405
2,10689,0.526405
3,10690,0.373432
4,10690,0.373432
...,...,...
2546,12364,0.977566
2547,12365,0.260521
2548,12366,0.283367
2549,12367,0.602721


In [45]:
test['target'].mean()

0.528922887190328

In [46]:
test.groupby(['Opportunity_ID'], as_index=False)['target'].mean()[['Opportunity_ID', 'target']]

,Opportunity_ID,target
0,10689,0.526405
1,10690,0.373432
2,10691,0.747902
3,10692,0.891873
4,10693,0.943449
...,...,...
1562,12364,0.977566
1563,12365,0.260521
1564,12366,0.283367
1565,12367,0.602721


In [47]:
test =test.groupby(['Opportunity_ID'], as_index=False)['target'].mean()[['Opportunity_ID', 'target']]

In [48]:
test['target'].mean()

0.5011659773311415

In [49]:
test[['Opportunity_ID', 'target']].to_csv("subidas/LGBM_label_completo.csv", index=False)